# Basic Testing of FabSim3 using a Jupyter notebook

This short tutorial shows you how you can use FabSim3 from inside a Jupyter notebook, and demonstrates how you can run simple jobs both locally, and using the ARCHER2 supercomputer.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

## Setting up FabSim3

We will assume you have FabSim3 installed. If not, you can find instructions [here](https://fabsim3.readthedocs.io/en/latest/). 

To test whether FabSim3 works and try out some simple commands, you will need to install the FabDummy plugin. Simply run 

```
fabsim localhost install_plugin:FabDummy
```

This will install the plugin in `FabSim3/plugins/FabDummy`.

You will now need to configure the FabSim3 script, config and `machines_user.yml` file.

**machines_user.yml**

This file is located in `FabSim3/fabsim/deploy`. Open it. This is an important file that lets you configure your run environment for various machines, both your `localhost` as well as remote machines. My `localhost` machine is posted below. In yours, also add the last line that says `ade_exec_school`.

```python
localhost:
  username: wouter
  partition_name: "standard"
  qos_name: "short"
  runtime: 10
  nodes: 1
  ranks: 1
  cores_per_rank: 1
  home_path_template: "/home/wouter/FabSim3/localhost_exe"
```

We will set the following FabSim3 flags:

In [2]:
# FabSim3 arguments
ARGUMENTS = 'dummy_test'
# machine to run ensemble on
MACHINE = "localhost"

### FabSim3-Python interface

FabSim3 is a command line tool, but it features a Python interface that is simply a shell over the commandline tools. It is stored locally here. To import it use

In [3]:
# Import the FabSim3 commandline interface
import fabsim3_cmd_api as fab

We are now ready to submit the ensemble, in this case to `localhost`. The command `fab.fabsim()` can be used to run any FabSim3 command. It takes three arguments:

* command: the name of the Fabsim3 command you seek to use.
* arguments: a string containing the arguments of the command as you would type them on the command line.
* machine: a string containing the label of the machine you wish to use, e.g. 'localhost' or 'archer2'.

**Note**: if you get some type of ssh error, you might have to ssh into the (remote) machine once via the command line before executing `fab.fabsim`.

We now proceed to run a single dummy job, using the `dummy` command.

In [4]:
fab.fabsim('dummy', ARGUMENTS, MACHINE)

Executing fabsim localhost dummy:dummy_test


This will check every minute on the status of the jobs on the remote host, and sleep otherwise, halting further execution of the script. On the localhost this command doesn't do anything.

In [5]:
# wait for job to complete
fab.wait(machine=MACHINE)

True

`fab.verify` will execute the `verify_last_ensemble` subroutine to see if the output file `target_filename` (`output.csv` in our case) for each run in the `SWEEP` directory is present in the corresponding FabSim3 results directory. Returns a boolean flag. `fab.verify` will also call the FabSim `fetch_results` method, which actually retrieves the results from the (remote) host. 

So, if you want to just get the results without verifying the presence of output files, call `fab.fetch_results(machine=MACHINE)` instead. However, if something went wrong on the (remote) host, this will cause an error later on since not all required output files will be transfered on the EasyVVUQ `WORK_DIR`.

`fab.get_uq_samples` copies the samples from the (local) FabSim results directory to the (local) EasyVVUQ campaign directory. It will not delete the results from the FabSim results directory. If you want to save space, you can delete the results on the FabSim side (see `results` directory in your FabSim home directory). You can also call `fab.clear_results(machine, name_results_dir)` to remove a specific FabSim results directory on a given machine.

Once all output files are copied to the EasyVVUQ run directories, we no longer need the (now duplicate) results in the FabSim3 directory. You can remove these via the command below, once you change the path from my FabSim3 directory to yours.

In [29]:
# clear results in Fabsim directory, change path to your FabSim3 directory
fab.clear_results("localhost", '/home/wouter/FabSim3/results/')  # localhost
# fab.clear_results("archer2", '/work/e723/e723/edeling/FabSim/')  # archer2

Executing fabsim archer2 clear_results:/work/e723/e723/edeling/FabSim/


/home/wouter/anaconda3/lib/python3.9/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


## Assigment: repeat on Archer2

If you have aquired access to Archer2, you can redo this exercise, except run the ensemble on Archer2 instead on your laptop. To do so you you will need to:

* Copy the advection-diffusion model to a location in Archer2. From within this path, you can do so via `scp -r advection_diffusion_model/ <username>@login.archer2.ac.uk:~/`, replacing `<username>` with your archer2 user name. This will copy `advection_diffusion_model/*` to your Archer2 home directory.
* Configure Archer2 in your `machines_user.yml` file. My file is posted below. Copy it, replace the archer2 entry in your `machine_user.yml`, and modify where needed, replacing my username with yours, and my project (`e723`) with yours (`ta171`). Note that for larger models, this is also where you can request more than 1 compute node.

```python
archer2:
  username: "edeling"
  project: "e723"
  budget: "e723-edeling"
  manual_ssh: true
  remote: "archer2"
  job_wall_time: '0-00:10:00'
  partition_name: "standard"
  ade_exec_school : "/home/e723/e723/edeling/advection_diffusion_model/advection_diffusion.py"
  qos_name: "short"
  runtime: 10
  nodes: 1
  ranks: 1
  cores_per_rank: 1
  run_prefix_commands: ["export PYTHONUSERBASE=/work/$project/$project/$username/.local", "export PATH=$PYTHONUSERBASE/bin:$PATH", "export PYTHONPATH=$PYTHONUSERBASE/lib/python3.9/site-packages:$PYTHONPATH"]
  modules:
        loaded: ["cray-python/3.9.13.1"]
        unloaded: ["cray-python"]
```

* Once Archer2 if configured properly, switching from local ensemble execution to execution on Archer2 is done by simply changing the `MACHINE` flag in this notebook from `"localhost"` to `"archer2"`.